In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml, make_blobs
np.random.seed(42)

def load_data(use_synthetic=False, n_samples=5000):
    if use_synthetic:
        X, y = make_blobs(n_samples=2000, centers=3, n_features=4, random_state=0)
        num_classes = 3
    else:
        mnist = fetch_openml("mnist_784", version=1, as_frame=False)
        X, y = mnist["data"] / 255.0, mnist["target"].astype(int)
        num_classes = 10
        X, y = X[:n_samples], y[:n_samples]  # speed
    y_oh = np.eye(num_classes)[y]
    X_train, X_test, y_train, y_test = train_test_split(X, y_oh, test_size=0.2, random_state=0)
    return X_train, X_test, y_train, y_test, num_classes

X_train, X_test, y_train, y_test, num_classes = load_data(use_synthetic=True)
X_train.shape, y_train.shape


((1600, 4), (1600, 3))

In [2]:
class TwoLayerNet:
    def __init__(self, input_dim, hidden_dim, output_dim, lr=0.1, reg=1e-4):
        self.lr, self.reg = lr, reg
        self.params = {
            "W1": 0.01 * np.random.randn(input_dim, hidden_dim),
            "b1": np.zeros((1, hidden_dim)),
            "W2": 0.01 * np.random.randn(hidden_dim, output_dim),
            "b2": np.zeros((1, output_dim)),
        }

    def forward(self, X):
        W1, b1, W2, b2 = self.params.values()
        z1 = X @ W1 + b1
        a1 = np.maximum(0, z1)  # ReLU
        scores = a1 @ W2 + b2
        scores -= scores.max(axis=1, keepdims=True)
        exp = np.exp(scores)
        probs = exp / exp.sum(axis=1, keepdims=True)
        cache = (X, z1, a1, probs)
        return probs, cache

    def loss(self, probs, y_true):
        N = y_true.shape[0]
        log_likelihood = -np.log(probs[y_true.astype(bool)])
        data_loss = log_likelihood.mean()
        reg_loss = 0.5 * self.reg * sum((p**2).sum() for k, p in self.params.items() if k.startswith("W"))
        return data_loss + reg_loss

    def backward(self, cache, probs, y_true):
        X, z1, a1, _ = cache
        N = X.shape[0]
        grads = {}
        dscores = (probs - y_true) / N
        grads["W2"] = a1.T @ dscores + self.reg * self.params["W2"]
        grads["b2"] = dscores.sum(axis=0, keepdims=True)
        da1 = dscores @ self.params["W2"].T
        dz1 = da1 * (z1 > 0)
        grads["W1"] = X.T @ dz1 + self.reg * self.params["W1"]
        grads["b1"] = dz1.sum(axis=0, keepdims=True)
        return grads

    def step(self, grads):
        for k in self.params:
            self.params[k] -= self.lr * grads[k]


In [3]:
def accuracy(model, X, y):
    probs, _ = model.forward(X)
    preds = probs.argmax(axis=1)
    targets = y.argmax(axis=1)
    return (preds == targets).mean()

def train(model, X_train, y_train, X_val, y_val, epochs=20, batch_size=64):
    N = X_train.shape[0]
    for epoch in range(1, epochs+1):
        idx = np.random.permutation(N)
        X_train, y_train = X_train[idx], y_train[idx]
        for i in range(0, N, batch_size):
            xb, yb = X_train[i:i+batch_size], y_train[i:i+batch_size]
            probs, cache = model.forward(xb)
            grads = model.backward(cache, probs, yb)
            model.step(grads)
        if epoch % 2 == 0 or epoch == 1:
            train_acc = accuracy(model, X_train, y_train)
            val_acc = accuracy(model, X_val, y_val)
            print(f"epoch {epoch}: train_acc={train_acc:.3f}, val_acc={val_acc:.3f}")

model = TwoLayerNet(input_dim=X_train.shape[1], hidden_dim=128, output_dim=num_classes, lr=0.1)
train(model, X_train, y_train, X_test, y_test, epochs=20, batch_size=64)


epoch 1: train_acc=0.999, val_acc=1.000
epoch 2: train_acc=1.000, val_acc=1.000
epoch 4: train_acc=1.000, val_acc=1.000
epoch 6: train_acc=1.000, val_acc=1.000
epoch 8: train_acc=1.000, val_acc=1.000
epoch 10: train_acc=1.000, val_acc=1.000
epoch 12: train_acc=1.000, val_acc=1.000
epoch 14: train_acc=1.000, val_acc=1.000
epoch 16: train_acc=1.000, val_acc=1.000
epoch 18: train_acc=1.000, val_acc=1.000
epoch 20: train_acc=1.000, val_acc=1.000


In [4]:
def numerical_grad(f, x, eps=1e-5):
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=["multi_index"], op_flags=["readwrite"])
    while not it.finished:
        idx = it.multi_index
        old = x[idx]
        x[idx] = old + eps
        plus = f(x)
        x[idx] = old - eps
        minus = f(x)
        x[idx] = old
        grad[idx] = (plus - minus) / (2 * eps)
        it.iternext()
    return grad

# Check W1 on a very small batch
mini_X, mini_y = X_train[:5], y_train[:5]
model = TwoLayerNet(input_dim=X_train.shape[1], hidden_dim=5, output_dim=num_classes, lr=0.1, reg=0.0)
probs, cache = model.forward(mini_X)
def loss_W1(W1):
    model.params["W1"] = W1
    p, c = model.forward(mini_X)
    return model.loss(p, mini_y)
grads = model.backward(cache, probs, mini_y)
num_g = numerical_grad(loss_W1, model.params["W1"].copy())
print("grad diff:", np.linalg.norm(grads["W1"] - num_g) / (np.linalg.norm(grads["W1"]) + np.linalg.norm(num_g)))


grad diff: 3.0785598113158576e-10
